In [1]:
!python3.6  -m pip install tensorflow-gpu==2.0.
!python3.6 -m pip install keras==2.3.1
!python3.6 -m pip install  'h5py==2.10.0' --force-reinstall

  Using cached h5py-2.10.0-cp36-cp36m-manylinux1_x86_64.whl (2.9 MB)
  Using cached numpy-1.19.5-cp36-cp36m-manylinux2010_x86_64.whl (14.8 MB)
  Using cached six-1.16.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: h5py
    Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0


In [2]:
!python3.6 -m pip install scikit-image


In [3]:
!python3.6  -m pip install tensorflow==2.0.0

In [4]:
from __future__ import absolute_import
from __future__  import division
from __future__ import print_function
import tensorflow as tf
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import cv2
import numpy as np
import os
from PIL import Image
from io import BytesIO
import time
import h5py


In [5]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1730894325101556574
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 1311654346841953654
physical_device_desc: "device: XLA_CPU device"
]


In [6]:
!python3.6 -m pip  freeze


absl-py==0.15.0
astor==0.8.1
astunparse==1.6.3
backcall==0.2.0
cached-property==1.5.2
cachetools==4.2.4
certifi==2023.11.17
charset-normalizer==2.0.12
clang==5.0
contextlib2==21.6.0
cycler==0.11.0
Cython==3.0.6
dataclasses==0.8
decorator==4.4.2
entrypoints==0.4
flatbuffers==1.12
gast==0.2.2
google-auth==1.35.0
google-auth-oauthlib==0.4.6
google-pasta==0.2.0
grpcio==1.48.2
h5py==2.10.0
idna==3.6
imageio==2.15.0
importlib-metadata==4.8.3
importlib-resources==5.4.0
ipykernel==5.5.6
ipython==7.16.3
ipython-genutils==0.2.0
jedi==0.17.2
jupyter-client==7.1.2
jupyter-core==4.9.2
Keras==2.3.1
Keras-Applications==1.0.8
Keras-Preprocessing==1.1.2
kiwisolver==1.3.1
lvis==0.5.3
lxml==4.9.3
Markdown==3.3.7
matplotlib==3.3.4
nest-asyncio==1.5.8
networkx==2.5.1
numpy==1.19.5
oauthlib==3.2.2
object-detection @ file:///opt/models/research
opencv-python==4.8.1.78
opt-einsum==3.3.0
pandas==1.1.5
parso==0.7.1
pexpect==4.9.0
pickleshare==0.7.5
Pillow==8.4.0
pkg_resources==0.0.0
prompt-toolkit==3.0.36
proto

In [29]:
def mamon_videoFightModel2(tf,wight='mamonbest947oscombo.h5'):
    layers = tf.keras.layers
    models = tf.keras.models
    losses = tf.keras.losses
    optimizers = tf.keras.optimizers
    metrics = tf.keras.metrics
    num_classes = 2
    cnn = models.Sequential()
    #cnn.add(base_model)

    input_shapes=(160,160,3)
    np.random.seed(1234)
    vg19 = tf.keras.applications.vgg19.VGG19
    base_model = vg19(include_top=False,weights='imagenet',input_shape=(160, 160,3))
    # Freeze the layers except the last 4 layers
    #for layer in base_model.layers:
    #    layer.trainable = False

    cnn = models.Sequential()
    cnn.add(base_model)
    cnn.add(layers.Flatten())
    model = models.Sequential()

    model.add(layers.TimeDistributed(cnn,  input_shape=(30, 160, 160, 3)))
    model.add(layers.LSTM(30 , return_sequences= True))

    model.add(layers.TimeDistributed(layers.Dense(90)))
    model.add(layers.Dropout(0.1))

    model.add(layers.GlobalAveragePooling1D())

    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.3))

    model.add(layers.Dense(num_classes, activation="sigmoid"))

    adam = optimizers.Adam(lr=0.0005, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.load_weights(wight)
    rms = optimizers.RMSprop()

    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=["accuracy"])

    return model


In [30]:
import numpy as np
from skimage.transform import resize
import sys
#reload(sys)
#sys.setdefaultencoding("utf-8")
np.random.seed(1234)
model22 = mamon_videoFightModel2(tf)


: 

In [9]:
model22._make_predict_function()


In [10]:
def video_mamonreader(cv2,filename):
    frames = np.zeros((30, 160, 160, 3), dtype=np.float)
    i=0
    print(frames.shape)
    vc = cv2.VideoCapture(filename)
    if vc.isOpened():
        rval , frame = vc.read()

   
        i +=1
        print("reading video******",filename)
        while i < 30:
            if frame is None:
                print("frame is none")
                break  # Break if there are no more frames
            rval, frame = vc.read()
            frm = resize(frame,(160,160,3))
            frm = np.expand_dims(frm,axis=0)
            if(np.max(frm)>1):
                frm = frm/255.0
            frames[i][:] = frm
            i +=1
    else:
        rval = False
    return frames

In [11]:
def pred_fight(model,video,acuracy=0.9):
    pred_test = model.predict(video)
    if pred_test[0][1] >=acuracy:
        return True , pred_test[0][1]
    else:
        return False , pred_test[0][1]

In [12]:
def main_fight(vidoss):
    vid = video_mamonreader(cv2,vidoss)
    datav = np.zeros((1, 30, 160, 160, 3), dtype=np.float)
    datav[0][:][:] = vid
    millis = int(round(time.time() * 1000))
    print(millis)
    f , precent = pred_fight(model22,datav,acuracy=0.65)
    millis2 = int(round(time.time() * 1000))
    print(millis2)
    res_mamon = {'fight':f , 'precentegeoffight':str(precent)}
    res_mamon['processing_time'] =  str(millis2-millis)
    return res_mamon

In [26]:
res = main_fight('hospital.mp4')



(30, 160, 160, 3)
1706031073994
1706031078151


In [27]:
res

{'fight': True, 'precentegeoffight': '0.6639507', 'processing_time': '4157'}

In [15]:
res1 = main_fight("hospital.mp4")

(30, 160, 160, 3)
1706030521475
1706030527188


In [16]:
res1

{'fight': True, 'precentegeoffight': '0.6639507', 'processing_time': '5713'}